In [3]:
import boto3
import time
import pandas
import os
import awswrangler as wr

from dotenv import find_dotenv, load_dotenv

# find .env automagically by walking up directories until it's found
dotenv_path = find_dotenv()
load_dotenv(dotenv_path)

ACCESS_KEY_ID = os.environ.get('AWS_ACCESS_KEY_ID')
SECRET_ACCESS_KEY = os.environ.get('AWD_SECRET_ACCESS_KEY')
TABLE_NAME = os.environ.get('TABLE_NAME')
DATABASE_NAME = os.environ.get('DATABASE_NAME')
RESULT_OUTPUT_LOCATION = os.environ.get('RESULT_OUTPUT_LOCATION')

In [9]:
CLIENT = boto3.client(
    "athena", 
    region_name="us-east-1", 
    aws_access_key_id=ACCESS_KEY_ID, 
    aws_secret_access_key=SECRET_ACCESS_KEY
)

In [16]:
def get_num_rows():
    query = f"SELECT * from {DATABASE_NAME}.{TABLE_NAME} LIMIT 10;"
    response = CLIENT.start_query_execution(
        QueryString=query,
        ResultConfiguration={"OutputLocation": RESULT_OUTPUT_LOCATION}
    )

    return response["QueryExecutionId"]

In [12]:
def has_query_succeeded(execution_id):
    state = "RUNNING"
    max_execution = 5

    while max_execution > 0 and state in ["RUNNING", "QUEUED"]:
        max_execution -= 1
        response = CLIENT.get_query_execution(QueryExecutionId=execution_id)
        if (
            "QueryExecution" in response
            and "Status" in response["QueryExecution"]
            and "State" in response["QueryExecution"]["Status"]
        ):
            state = response["QueryExecution"]["Status"]["State"]
            if state == "SUCCEEDED":
                return True

        time.sleep(30)

    return False

In [17]:
execution_id = get_num_rows()
print(f"Get Num Rows execution id: {execution_id}")

Get Num Rows execution id: 6fde92ca-5e29-4c18-b077-c8233dc16c44


In [18]:
def get_query_results(execution_id):
    response = CLIENT.get_query_results(
        QueryExecutionId=execution_id
    )

    results = response['ResultSet']['Rows']
    return results

In [20]:
query_status = has_query_succeeded(execution_id=execution_id)
print(f"Query state: {query_status}")

# 6. Query Results
print(get_query_results(execution_id=execution_id))

Query state: True
[{'Data': [{'VarCharValue': 'imo_number'}, {'VarCharValue': 'co_emissions_from_all_voyages_which_departed_from_ports_under_a_ms_jurisdiction_m_tonnes_'}, {'VarCharValue': 'total_fuel_consumption_m_tonnes_'}, {'VarCharValue': 'co_emissions_from_all_voyages_to_ports_under_a_ms_jurisdiction_m_tonnes_'}, {'VarCharValue': 'co_emissions_which_occurred_within_ports_under_a_ms_jurisdiction_at_berth_m_tonnes_'}, {'VarCharValue': 'verifier_country'}, {'VarCharValue': 'verifier_accreditation_number'}, {'VarCharValue': 'verifier_city'}, {'VarCharValue': 'verifier_address'}, {'VarCharValue': 'total_co_emissions_m_tonnes_'}, {'VarCharValue': 'verifier_nab'}, {'VarCharValue': 'doc_expiry_date'}, {'VarCharValue': 'doc_issue_date'}, {'VarCharValue': 'reporting_period'}, {'VarCharValue': 'ship_type'}, {'VarCharValue': 'name'}, {'VarCharValue': 'verifier_name'}, {'VarCharValue': 'co_emissions_from_all_voyages_between_ports_under_a_ms_jurisdiction_m_tonnes_'}, {'VarCharValue': 'technical

In [6]:
df = wr.athena.read_sql_query(f"SELECT * FROM {TABLE_NAME}", database=DATABASE_NAME)

In [7]:
df.shape

(44357, 22)

In [8]:
df.head()

,imo_number,co_emissions_from_all_voyages_which_departed_from_ports_under_a_ms_jurisdiction_m_tonnes_,total_fuel_consumption_m_tonnes_,co_emissions_from_all_voyages_to_ports_under_a_ms_jurisdiction_m_tonnes_,co_emissions_which_occurred_within_ports_under_a_ms_jurisdiction_at_berth_m_tonnes_,verifier_country,verifier_accreditation_number,verifier_city,verifier_address,total_co_emissions_m_tonnes_,...,doc_issue_date,reporting_period,ship_type,name,verifier_name,co_emissions_from_all_voyages_between_ports_under_a_ms_jurisdiction_m_tonnes_,technical_efficiency,port_of_registry,technical_efficiency_type,technical_efficiency_value
0,9595890,0.00,0.00,0.00,0.00,Greece,1083,Kallithea,1 Sachtouri & 1 Navarinou Street,0.00,...,31/10/2022,2021,Bulk carrier,FEDERAL SKEENA,AMERICAN BUREAU OF SHIPPING HELLENIC SINGLE ME...,0.00,EIV (5.65 gCO₂/t·nm),Majuro,EIV,5.65
1,9363168,954.53,1124.58,2363.81,94.90,Greece,1190,Athens,"348 Syggrou Avenue,",3525.93,...,27/04/2022,2021,Bulk carrier,ATALANTE,HELLENIC LLOYD'S S.A.,112.70,EIV (7.34 gCO₂/t·nm),Valletta,EIV,7.34
2,9506291,0.00,22010.65,0.00,7605.30,France,4-0076,92937 PARIS LA DEFENSE,"Le Triangle de l'Arche 9, cours du Triangle",68669.01,...,22/04/2022,2021,Ro-pax ship,VOLCAN DE TINAMAR,Bureau Veritas Certification France,61063.67,EIV (24.64 gCO₂/t·nm),Las Palmas de Gran Canaria,EIV,24.64
3,9765562,11521.50,8080.71,13004.92,838.24,Japan,VS-21325-01,"Tokyo, 102-8567","4-7, Kioi-Cho, Chiyoda-Ku,",25580.11,...,06/05/2022,2021,Gas carrier,GAS STELLA,Nippon Kaiji Kyokai,215.44,EEDI (5.93 gCO₂/t·nm),PANAMA,EEDI,5.93
4,9841653,23.11,147.69,431.00,6.89,Italy,002O,Genova,Via Corsica 12,461.00,...,27/05/2021,2020,Bulk carrier,OKOLCHITSA,RINA Services Spa,0.00,EEDI (4.58 gCO₂/t·nm),Valletta,EEDI,4.58


In [9]:
df.dtypes

imo_number                                                                                     Int64
co_emissions_from_all_voyages_which_departed_from_ports_under_a_ms_jurisdiction_m_tonnes_    float64
total_fuel_consumption_m_tonnes_                                                             float64
co_emissions_from_all_voyages_to_ports_under_a_ms_jurisdiction_m_tonnes_                     float64
co_emissions_which_occurred_within_ports_under_a_ms_jurisdiction_at_berth_m_tonnes_          float64
verifier_country                                                                              string
verifier_accreditation_number                                                                 string
verifier_city                                                                                 string
verifier_address                                                                              string
total_co_emissions_m_tonnes_                                                               

In [17]:
import pandas as pd

In [64]:
df['doc_expiry_date'].max()

Timestamp('2023-04-29 00:00:00')

In [20]:
df['doc_expiry_date'] = pd.to_datetime(df['doc_expiry_date'], dayfirst=True)

In [22]:
df['doc_expiry_date'].value_counts()

2023-06-30    11826
2021-06-30    11349
2022-06-30    11333
2020-06-30     9849
Name: doc_expiry_date, dtype: int64

In [26]:
df['doc_issue_date'] = pd.to_datetime(df['doc_issue_date'], dayfirst=True)

In [67]:
df['emissions_within_ports_at_berth']

0           0.00
1          94.90
2        7605.30
3         838.24
4           6.89
          ...   
44352     461.15
44353     175.27
44354     394.08
44355     181.20
44356      75.22
Name: emissions_within_ports_at_berth, Length: 44357, dtype: float64

In [23]:
df['doc_issue_date'].value_counts()

30/04/2020    1526
30/04/2019     841
29/04/2022     825
30/04/2022     773
29/04/2020     739
              ... 
25/07/2020       1
31/01/2021       1
22/12/2022       1
06/08/2022       1
14/10/2021       1
Name: doc_issue_date, Length: 1038, dtype: Int64

In [27]:
df['doc_issue_date'].value_counts()

2020-04-30    1526
2019-04-30     841
2022-04-29     825
2022-04-30     773
2020-04-29     739
              ... 
2021-09-28       1
2021-10-22       1
2019-03-13       1
2019-02-11       1
2021-10-14       1
Name: doc_issue_date, Length: 1038, dtype: int64

In [28]:
df.dtypes

imo_number                                                                                            Int64
co_emissions_from_all_voyages_which_departed_from_ports_under_a_ms_jurisdiction_m_tonnes_           float64
total_fuel_consumption_m_tonnes_                                                                    float64
co_emissions_from_all_voyages_to_ports_under_a_ms_jurisdiction_m_tonnes_                            float64
co_emissions_which_occurred_within_ports_under_a_ms_jurisdiction_at_berth_m_tonnes_                 float64
verifier_country                                                                                     string
verifier_accreditation_number                                                                        string
verifier_city                                                                                        string
verifier_address                                                                                     string
total_co_emissions_m_tonnes_

In [29]:
df['reporting_period'].value_counts()

2021    11826
2019    11349
2020    11333
2018     9849
Name: reporting_period, dtype: Int64

In [31]:
df[df['reporting_period'] == 2018]['total_co_emissions_m_tonnes_'].sum()

121987203.25999999

In [35]:
df[['name', 'ship_type', 'technical_efficiency_type', 'technical_efficiency_value']]

,name,ship_type,technical_efficiency_type,technical_efficiency_value
0,FEDERAL SKEENA,Bulk carrier,EIV,5.65
1,ATALANTE,Bulk carrier,EIV,7.34
2,VOLCAN DE TINAMAR,Ro-pax ship,EIV,24.64
3,GAS STELLA,Gas carrier,EEDI,5.93
4,OKOLCHITSA,Bulk carrier,EEDI,4.58
...,...,...,...,...
44352,ESSEN EXPRESS,Container ship,EEDI,10.3
44353,NORDSCHELDE,Bulk carrier,EIV,6.19
44354,THUN LONDON,Chemical tanker,EEDI,6.86
44355,SEA HOPE,Bulk carrier,EIV,6.91


In [45]:
df[df['name'] == 'BEATE OLDENDORFF'][['reporting_period' ,'technical_efficiency_value']]

,reporting_period,technical_efficiency_value
44356,2021,3.49


In [57]:
df.loc[df['technical_efficiency_value'] == '', 'technical_efficiency_value'] = '0'

In [50]:
df['technical_efficiency_value'] = df['technical_efficiency_value'].fillna('0')

In [58]:
df['technical_efficiency_value'] = df['technical_efficiency_value'].astype(float)

In [62]:
df.groupby('ship_type')['technical_efficiency_value'].mean().reset_index()

,ship_type,technical_efficiency_value
0,Bulk carrier,10.457462
1,Chemical tanker,8.286671
2,Combination carrier,13.617143
3,Container ship,17.100986
4,Container/ro-ro cargo ship,10.416667
5,Gas carrier,11.887620
6,General cargo ship,15.909757
7,LNG carrier,15.048695
8,Oil tanker,5.606511
9,Other ship types,13.746337
